In [1]:
import pandas as pd
import numpy as np

In [81]:
df_detailed_listings = pd.read_csv("data/eustartup_listings_modified.csv")

# cleaning weird text unicodings
df_detailed_listings['category'] = df_detailed_listings['category'].str.replace('\xa0',' ', regex=False)

### Adding data from industry reports to the main dataframe

In [82]:
# adding data from EY industry reports on number of startups with financing
df_ey_startups_num = pd.read_excel("data/Startups_AT_reports.xlsx", sheet_name="Startups with financing").loc[:, :'Number of startups that received financing']

# remove the space + "(" + everything until the closing ")"
df_ey_startups_num['Region'] = df_ey_startups_num['Region'].str.replace(r' \([^)]+\)', '', regex=True)
df_ey_startups_num['Year'] = df_ey_startups_num['Year'].apply(lambda x: f"startups_with_financing_{x}")

# wide format: columns = Year
df_ey_startups_num = df_ey_startups_num[['Year', 'Region', 'Number of startups that received financing']].pivot(index=['Region'], columns='Year', values='Number of startups that received financing').reset_index()

df_ey_startups_num

Year,Region,startups_with_financing_2021,startups_with_financing_2022,startups_with_financing_2023,startups_with_financing_2024
0,Carinthia,0,0,7,3
1,Lower Austria,6,11,13,15
2,Salzburg,3,4,11,4
3,Styria,12,15,22,15
4,Tyrol,5,6,9,9
5,Upper Austria,17,20,16,10
6,Vienna,71,77,93,80
7,Vorarlberg,2,1,2,2


In [83]:
# adding ranking with number of startups columns from different industry reports

# merging the dataframes in a temporary df
df_detailed_listings_temp = df_detailed_listings.merge(df_ey_startups_num, how='left', left_on=['region'], right_on=['Region'])

# compute rank by number of startups financed (1 being the highest rank):
# the higher the number of startups financed in the closest year to foundation is - the higher the rank
# in case of tiebraker: the row with a higher number of financed startups in the following year wins
# final tiebreaker: more recently founded startup wins

# mask out pre-foundation years
for y in [2021, 2022, 2023, 2024]:
    df_detailed_listings_temp[f"startups_with_financing_{y}"] = df_detailed_listings_temp[f"startups_with_financing_{y}"].where(df_detailed_listings_temp["founded"] <= y, float("-inf"))

# create per-row sort-key tuples
def make_key(row):
    # pick the first year to compare
    start = max(int(row["founded"]), 2021)
    start = min(start, 2024)
    # grab that year + every later year up to 2024
    seq = []
    for y in range(start, 2025):
        seq.append(row[f"startups_with_financing_{y}"])
    # final tiebreaker: more recently founded wins
    seq.append(row["founded"])
    return tuple(seq)

df_detailed_listings_temp["sort_key"] = df_detailed_listings_temp.apply(make_key, axis=1)

# lexicographically sort by that tuple, highest first
df_detailed_listings_temp = (
    df_detailed_listings_temp
      .sort_values("sort_key", ascending=False)
      .reset_index(drop=True)
)

# turn each unique tuple into a rank
df_detailed_listings_temp["region_rank"] = (
    pd.factorize(df_detailed_listings_temp["sort_key"])[0] + 1
)

df_detailed_listings_temp = df_detailed_listings_temp.drop(columns="sort_key")

# merging with the main dataframe
df_detailed_listings = df_detailed_listings.merge(df_ey_startups_num, how='left', left_on=['region'], right_on=['Region']).merge(df_detailed_listings_temp[['name', 'region_rank']], how='left', on=['name']).drop(columns="Region")

df_detailed_listings

,name,link_startupeu,link_logo,website,city,region,tags,category,business_description,founded,age,total_funding,company_status,social_links,startups_with_financing_2021,startups_with_financing_2022,startups_with_financing_2023,startups_with_financing_2024,region_rank
0,Avdain,https://www.eu-startups.com/directory/avdain/,https://www.eu-startups.com/wp-content/uploads...,avdain.com,Vienna,Vienna,"company, startup, one person",Software & Analytics,Avdain is a enterprise that embodies a fusion ...,2020,5,No funding announced yet,Active,(),71.0,77.0,93.0,80.0,5
1,Popper Power GmbH,https://www.eu-startups.com/directory/popper-p...,https://www.eu-startups.com/wp-content/uploads...,www.popperpower.com,Vienna,Vienna,"ev, battery, bess, charging",Energy,Popper Power GmbH develops advanced energy sto...,2022,3,Between €500K-€ 1 million,Active,"('https://www.linkedin.com/company/86313916',)",71.0,77.0,93.0,80.0,3
2,Setter AI,https://www.eu-startups.com/directory/setter-ai/,https://www.eu-startups.com/wp-content/uploads...,https://www.trysetter.com,Wien,Vienna,"ai, ai agents, sales & marketing, ai saas, ai ...",Software & Analytics,Speed matters when you want more sales. That’s...,2024,1,Between €1-€100K,Active,(),71.0,77.0,93.0,80.0,2
3,SurveySensum,https://www.eu-startups.com/directory/surveyse...,https://www.eu-startups.com/wp-content/uploads...,https://www.surveysensum.com,vienna,Vienna,"customer feedback, customer experience",Software & Analytics,SurveySensum is a leading customer feedback pl...,2018,7,No funding announced yet,Active,('https://www.linkedin.com/company/SurveySensu...,71.0,77.0,93.0,80.0,7
4,Artypa,https://www.eu-startups.com/directory/artypa/,https://www.eu-startups.com/wp-content/uploads...,https://artypa.com,Vienna,Vienna,"ai content, ai productivity, ai generation",Software & Analytics,Traditional AI workflows often involve navigat...,2024,1,No funding announced yet,Active,(),71.0,77.0,93.0,80.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,Ease-Link,https://www.eu-startups.com/directory/ease-link/,https://www.eu-startups.com/wp-content/uploads...,http://easelink.com/,Graz,Styria,"automotive, industrial automation",Mobility,"Ease-Link GmbH develops MATRIX CHARGING®, the ...",2015,10,Between €2.5 million-5 million,NaN,(),12.0,15.0,22.0,15.0,29
514,Healcloud,https://www.eu-startups.com/directory/healcloud/,https://www.eu-startups.com/wp-content/uploads...,http://www.healcloud.com/,Vienna,Vienna,"big data, health care",Software & Analytics,Healcloud is a European health tech enabler to...,2015,10,Between €2.5 million-5 million,NaN,(),71.0,77.0,93.0,80.0,10
515,Rendity,https://www.eu-startups.com/directory/rendity/,https://www.eu-startups.com/wp-content/uploads...,https://rendity.com/,Vienna,Vienna,"finance, financial services, real estate",FinTech/InsurTech,Rendity enables real estate investments for ev...,2015,10,Between €1 million-€ 2.5 million,NaN,(),71.0,77.0,93.0,80.0,10
516,Ubiq.ai,https://www.eu-startups.com/directory/ubiq-ai/,https://www.eu-startups.com/wp-content/uploads...,https://www.ubiq.ai/,Vienna,Vienna,"autonomous vehicles, car sharing, internet, la...",Mobility,Ubiq is shaping the future of urban mobility b...,2015,10,Between €500K-€ 1 million,NaN,(),71.0,77.0,93.0,80.0,10


In [84]:
# adding data from EY industry reports on number of deals by sector
df_ey_deals_by_sec = pd.read_excel("data/Startups_AT_reports.xlsx", sheet_name="Deals by sector").loc[:, :'deals']

# rename the year column
df_ey_deals_by_sec['year'] = df_ey_deals_by_sec['year'].apply(lambda x: f"deals_by_sector_{x}")

# wide format: columns = year
df_ey_deals_by_sec = df_ey_deals_by_sec.pivot(index=['sector'], columns='year', values='deals').reset_index()

# rename weird texting in sector names
df_ey_deals_by_sec['sector'] = df_ey_deals_by_sec['sector'].str.replace('\xa0',' ', regex=False)
df_ey_deals_by_sec['sector'] = df_ey_deals_by_sec['sector'].str.replace('‑','-')

df_ey_deals_by_sec

year,sector,deals_by_sector_2021,deals_by_sector_2022,deals_by_sector_2023,deals_by_sector_2024
0,AgTech/FoodTech,NaN,4.0,3.0,3.0
1,ClimateTech/GreenTech/CleanTech,NaN,NaN,4.0,5.0
2,ConstructionTec/Green Building,NaN,NaN,3.0,NaN
3,Education,5.0,NaN,6.0,4.0
4,Energy,NaN,NaN,5.0,15.0
5,FinTech/InsurTech,5.0,11.0,11.0,4.0
6,Hardware,8.0,7.0,4.0,5.0
7,Health,4.0,14.0,18.0,19.0
8,Media & Entertainment,10.0,6.0,6.0,NaN
9,Mobility,8.0,11.0,12.0,6.0


In [85]:
# adding ranking with number of deals by sector from EY reports

# getting the old dataframe with non-singular categories
df_detailed_listings_old = pd.read_csv("data/eustartup_listings_adjusted.csv")
df_detailed_listings_old['mapped_tags'] = df_detailed_listings_old['mapped_tags'].str.replace('\xa0',' ', regex=False)

# merging the dataframes in a temporary df
# split + explode
df_detailed_listings_temp = df_detailed_listings_old.assign(keys_list=df_detailed_listings_old['mapped_tags'].str.split(','))
df_detailed_listings_temp_exploded = df_detailed_listings_temp.explode('keys_list')

# merge
df_detailed_listings_temp = df_detailed_listings_temp_exploded.merge(
    df_ey_deals_by_sec,
    left_on='keys_list',
    right_on='sector',
    how='left'
).drop(columns=['keys_list', 'sector'])

# build an agg-dict to sum up the TS columns but not the rest:
agg = {
    'link':'first', 
    'category':'first', 
    'based_in':'first', 
    'tags':'first', 
    'founded':'first',
    'business_name':'first', 
    'logo_link':'first', 
    'long_business_description':'first',
    'business_description':'first', 
    'total_funding':'first', 
    'website':'first', 
    'company_status':'first',
    'social_links':'first', 
    'based_in_norm':'first', 
    'tags_norm':'first', 
    'mapped_tags':'first',
    'total_funding_norm':'first', 
    'deals_by_sector_2021':'sum', 
    'deals_by_sector_2022':'sum',
    'deals_by_sector_2023':'sum', 
    'deals_by_sector_2024':'sum'
}

df_detailed_listings_temp = df_detailed_listings_temp.groupby('name', as_index=False).agg(agg)

# compute rank by number of startups financed (1 being the highest rank):
# the higher the number of startups financed in the closest year to foundation is - the higher the rank
# in case of tiebraker: the row with a higher number of financed startups in the following year wins
# final tiebreaker: more recently founded startup wins

# mask out pre-foundation years


In [86]:
for y in [2021, 2022, 2023, 2024]:
    df_detailed_listings_temp[f"deals_by_sector_{y}"] = df_detailed_listings_temp[f"deals_by_sector_{y}"].where(df_detailed_listings_temp["founded"] <= y, float("-inf"))

# create per-row sort-key tuples
def make_key(row):
    # pick the first year to compare
    start = max(int(row["founded"]), 2021)
    start = min(start, 2024)
    # grab that year + every later year up to 2024
    seq = []
    for y in range(start, 2025):
        seq.append(row[f"deals_by_sector_{y}"])
    # final tiebreaker: more recently founded wins
    seq.append(row["founded"])
    return tuple(seq)

df_detailed_listings_temp["sort_key"] = df_detailed_listings_temp.apply(make_key, axis=1)

# lexicographically sort by that tuple, highest first
df_detailed_listings_temp = (
    df_detailed_listings_temp
      .sort_values("sort_key", ascending=False)
      .reset_index(drop=True)
)

# turn each unique tuple into a rank
df_detailed_listings_temp["categories_rank"] = (
    pd.factorize(df_detailed_listings_temp["sort_key"])[0] + 1
)

df_detailed_listings_temp = df_detailed_listings_temp.drop(columns="sort_key")

# merging with the main dataframe
df_detailed_listings = df_detailed_listings.merge(df_ey_deals_by_sec, how='left', left_on=['category'], right_on=['sector']).merge(df_detailed_listings_temp[['name', 'categories_rank']], how='left', on=['name']).drop(columns="sector")

In [87]:
df_detailed_listings

,name,link_startupeu,link_logo,website,city,region,tags,category,business_description,founded,...,startups_with_financing_2021,startups_with_financing_2022,startups_with_financing_2023,startups_with_financing_2024,region_rank,deals_by_sector_2021,deals_by_sector_2022,deals_by_sector_2023,deals_by_sector_2024,categories_rank
0,Avdain,https://www.eu-startups.com/directory/avdain/,https://www.eu-startups.com/wp-content/uploads...,avdain.com,Vienna,Vienna,"company, startup, one person",Software & Analytics,Avdain is a enterprise that embodies a fusion ...,2020,...,71.0,77.0,93.0,80.0,5,5.0,33.0,58.0,53.0,215
1,Popper Power GmbH,https://www.eu-startups.com/directory/popper-p...,https://www.eu-startups.com/wp-content/uploads...,www.popperpower.com,Vienna,Vienna,"ev, battery, bess, charging",Energy,Popper Power GmbH develops advanced energy sto...,2022,...,71.0,77.0,93.0,80.0,3,NaN,NaN,5.0,15.0,181
2,Setter AI,https://www.eu-startups.com/directory/setter-ai/,https://www.eu-startups.com/wp-content/uploads...,https://www.trysetter.com,Wien,Vienna,"ai, ai agents, sales & marketing, ai saas, ai ...",Software & Analytics,Speed matters when you want more sales. That’s...,2024,...,71.0,77.0,93.0,80.0,2,5.0,33.0,58.0,53.0,11
3,SurveySensum,https://www.eu-startups.com/directory/surveyse...,https://www.eu-startups.com/wp-content/uploads...,https://www.surveysensum.com,vienna,Vienna,"customer feedback, customer experience",Software & Analytics,SurveySensum is a leading customer feedback pl...,2018,...,71.0,77.0,93.0,80.0,7,5.0,33.0,58.0,53.0,122
4,Artypa,https://www.eu-startups.com/directory/artypa/,https://www.eu-startups.com/wp-content/uploads...,https://artypa.com,Vienna,Vienna,"ai content, ai productivity, ai generation",Software & Analytics,Traditional AI workflows often involve navigat...,2024,...,71.0,77.0,93.0,80.0,2,5.0,33.0,58.0,53.0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,Ease-Link,https://www.eu-startups.com/directory/ease-link/,https://www.eu-startups.com/wp-content/uploads...,http://easelink.com/,Graz,Styria,"automotive, industrial automation",Mobility,"Ease-Link GmbH develops MATRIX CHARGING®, the ...",2015,...,12.0,15.0,22.0,15.0,29,8.0,11.0,12.0,6.0,101
514,Healcloud,https://www.eu-startups.com/directory/healcloud/,https://www.eu-startups.com/wp-content/uploads...,http://www.healcloud.com/,Vienna,Vienna,"big data, health care",Software & Analytics,Healcloud is a European health tech enabler to...,2015,...,71.0,77.0,93.0,80.0,10,5.0,33.0,58.0,53.0,164
515,Rendity,https://www.eu-startups.com/directory/rendity/,https://www.eu-startups.com/wp-content/uploads...,https://rendity.com/,Vienna,Vienna,"finance, financial services, real estate",FinTech/InsurTech,Rendity enables real estate investments for ev...,2015,...,71.0,77.0,93.0,80.0,10,5.0,11.0,11.0,4.0,40
516,Ubiq.ai,https://www.eu-startups.com/directory/ubiq-ai/,https://www.eu-startups.com/wp-content/uploads...,https://www.ubiq.ai/,Vienna,Vienna,"autonomous vehicles, car sharing, internet, la...",Mobility,Ubiq is shaping the future of urban mobility b...,2015,...,71.0,77.0,93.0,80.0,10,8.0,11.0,12.0,6.0,50


### Adding economic indicators from Statistic Austria to the main dataframe

In [ ]:
# adding data from Statistic Austria on population by region
df_sa_pop = pd.read_excel("data/table_2025-05-04_22-32-01.xlsx", sheet_name="Data Sheet 0")

# rename years
df_sa_pop.rename(columns={x: 'population_'+x for x in df_sa_pop.columns if x not in ['region']}, inplace=True)

# identify the population columns to reorder
pop_cols = [c for c in df_sa_pop.columns if c.startswith('population_')]
# sort by the integer year
pop_cols_sorted = sorted(pop_cols, key=lambda c: int(c.split('_')[1]))
# build new column order:
#    - first any columns not in pop_cols
#    - then the population columns in ascending year order
other_cols = [c for c in df_sa_pop.columns if c not in pop_cols]
new_order  = other_cols + pop_cols_sorted
df_sa_pop = df_sa_pop[new_order]

df_sa_pop

,region,population_2000,population_2001,population_2002,population_2003,population_2004,population_2005,population_2006,population_2007,population_2008,...,population_2014,population_2015,population_2016,population_2017,population_2018,population_2019,population_2020,population_2021,population_2022,population_2023
0,Burgenland,276083,276331,276497,276481,277387,278466,279590,280469,281891,...,287791,289262,291663,292160,292966,293861,295098,296704,299968,301495
1,Carinthia,560129,559745,559179,558319,558431,559093,559453,559829,559723,...,556637,558612,561099,560852,560787,561062,561647,563176,567196,569108
2,Lower Austria,1537266,1542033,1547023,1552896,1562913,1574536,1584525,1592500,1599645,...,1631157,1643001,1661109,1667630,1673607,1680588,1687651,1695013,1711341,1721067
3,Upper Austria,1371579,1375473,1380561,1385088,1391321,1397572,1402281,1405127,1407809,...,1431253,1444122,1460276,1469187,1477752,1486141,1493377,1500043,1515781,1526805
4,Salzburg,513853,515949,516764,517908,520745,523356,525300,525826,526315,...,536340,541439,547639,550976,553930,556627,559543,561416,565851,569911
5,Styria,1182684,1185275,1188561,1190339,1194368,1198543,1201495,1203132,1204262,...,1217861,1225187,1235582,1238067,1241228,1244474,1247030,1249278,1260016,1267269
6,Tyrol,669479,673595,677060,680705,685505,690668,695296,697863,700562,...,724878,732671,742590,748186,752262,755695,758901,761596,767795,773496
7,Vorarlberg,349257,351356,353635,355621,358521,361391,363389,364985,366397,...,376973,381000,386708,390296,393025,395949,398443,400469,404253,408399
8,Vienna,1551236,1562536,1582841,1600888,1620250,1641653,1656619,1665458,1674937,...,1781042,1814225,1853140,1877719,1892150,1903240,1915155,1923825,1960655,1993147


In [103]:
# adding data from Statistic Austria on added value and employment by region
df_sa_av_empl = pd.read_excel("data/table_2025-05-04_10-30-27.xlsx", sheet_name="Data Sheet 0")

# rename columns
df_sa_av_empl.rename(columns=str.lower, inplace=True)
df_sa_av_empl.columns = df_sa_av_empl.columns.str.replace(r'[; ]+', '_', regex=True)

# keep only columns with variables required
substrings = ['region', 'employees_total_hours_worked_(in_million)_', 'self-employees_total_hours_worked_(in_million)_',
              'gross_fixed_capital_formation_current_prices_in_million_euro_',
              'compensation_of_employees_current_prices_in_million_euro_',
              'gross_value_added_at_basic_prices_current_prices_in_million_euro_']
pattern = '|'.join(substrings)
df_sa_av_empl = df_sa_av_empl.filter(regex=pattern, axis=1).replace('-', '')

# identify the population columns to reorder
pop_cols = [c for c in df_sa_av_empl.columns if "_" in c]
# sort by the integer year
pop_cols_sorted = sorted(pop_cols, key=lambda c: int(c.split('_')[-1]))
# build new column order:
#    - first any columns not in pop_cols
#    - then the population columns in ascending year order
other_cols = [c for c in df_sa_av_empl.columns if c not in pop_cols]
new_order  = other_cols + pop_cols_sorted
df_sa_av_empl = df_sa_av_empl[new_order]

df_sa_av_empl

,region,gross_fixed_capital_formation_current_prices_in_million_euro_2000,compensation_of_employees_current_prices_in_million_euro_2000,gross_value_added_at_basic_prices_current_prices_in_million_euro_2000,gross_fixed_capital_formation_current_prices_in_million_euro_2001,compensation_of_employees_current_prices_in_million_euro_2001,gross_value_added_at_basic_prices_current_prices_in_million_euro_2001,gross_fixed_capital_formation_current_prices_in_million_euro_2002,compensation_of_employees_current_prices_in_million_euro_2002,gross_value_added_at_basic_prices_current_prices_in_million_euro_2002,...,gross_value_added_at_basic_prices_current_prices_in_million_euro_2020,gross_fixed_capital_formation_current_prices_in_million_euro_2021,compensation_of_employees_current_prices_in_million_euro_2021,gross_value_added_at_basic_prices_current_prices_in_million_euro_2021,gross_fixed_capital_formation_current_prices_in_million_euro_2022,compensation_of_employees_current_prices_in_million_euro_2022,gross_value_added_at_basic_prices_current_prices_in_million_euro_2022,gross_fixed_capital_formation_current_prices_in_million_euro_2023,compensation_of_employees_current_prices_in_million_euro_2023,gross_value_added_at_basic_prices_current_prices_in_million_euro_2023
0,Burgenland,1257.4,2342.3,4369.4,1306.4,2373.3,4494.9,1259.2,2422.4,4642.2,...,7855.8,2989.5,4700.5,8424.5,3139.8,5027.6,9242.9,,,9871.9
1,Lower Austria,8461.7,16082.0,29999.9,8334.4,16311.0,30317.0,7913.4,16621.7,30731.8,...,53106.3,16521.0,30967.5,56593.2,18016.2,33267.2,62488.8,,,65741.8
2,Vienna,15696.0,28688.9,50349.8,15015.4,29376.8,52098.5,14741.0,30152.3,54042.9,...,85741.0,25681.3,53545.9,92300.6,27368.7,57710.0,99178.9,,,106580.3
3,Carinthia,3075.7,6023.6,11167.0,3240.2,6100.2,11476.1,3009.8,6180.4,11742.2,...,18962.8,6131.3,10885.1,20147.8,6914.0,11646.9,22507.6,,,23923.4
4,Styria,6858.5,13424.3,24465.1,7102.1,13626.0,25348.6,6738.3,13826.4,25611.6,...,43866.4,14076.4,26796.6,46553.2,14962.4,28756.2,51589.0,,,54981.9
5,Upper Austria,8401.8,17236.6,31201.7,8515.5,17691.4,32915.8,8554.6,17889.7,33504.6,...,59570.8,18488.7,35089.1,63495.2,19449.8,37549.8,69346.3,,,73438.6
6,Salzburg,3691.9,7023.1,13338.7,3850.9,7186.6,13769.9,3703.2,7355.1,14133.2,...,25142.2,7324.8,13432.9,26308.1,7997.0,14880.6,30282.9,,,32590.2
7,Tyrol,5002.4,8196.9,15876.9,4979.7,8430.9,16627.6,5055.4,8552.1,17118.8,...,29870.6,9235.9,16733.0,30217.0,10525.6,18590.4,35249.9,,,37676.2
8,Vorarlberg,2152.2,4515.8,8555.4,2335.6,4667.5,8829.4,2193.6,4725.4,9082.2,...,16285.8,4663.6,9142.7,18367.3,5747.0,9865.0,21348.4,,,20031.3


In [104]:
# merge up all the dataframes
df_detailed_listings = df_detailed_listings.merge(df_sa_pop, how='left', on='region').merge(df_sa_av_empl, how='left', on='region')

In [105]:
df_detailed_listings

,name,link_startupeu,link_logo,website,city,region,tags,category,business_description,founded,...,gross_value_added_at_basic_prices_current_prices_in_million_euro_2020,gross_fixed_capital_formation_current_prices_in_million_euro_2021,compensation_of_employees_current_prices_in_million_euro_2021,gross_value_added_at_basic_prices_current_prices_in_million_euro_2021,gross_fixed_capital_formation_current_prices_in_million_euro_2022,compensation_of_employees_current_prices_in_million_euro_2022,gross_value_added_at_basic_prices_current_prices_in_million_euro_2022,gross_fixed_capital_formation_current_prices_in_million_euro_2023,compensation_of_employees_current_prices_in_million_euro_2023,gross_value_added_at_basic_prices_current_prices_in_million_euro_2023
0,Avdain,https://www.eu-startups.com/directory/avdain/,https://www.eu-startups.com/wp-content/uploads...,avdain.com,Vienna,Vienna,"company, startup, one person",Software & Analytics,Avdain is a enterprise that embodies a fusion ...,2020,...,85741.0,25681.3,53545.9,92300.6,27368.7,57710.0,99178.9,,,106580.3
1,Popper Power GmbH,https://www.eu-startups.com/directory/popper-p...,https://www.eu-startups.com/wp-content/uploads...,www.popperpower.com,Vienna,Vienna,"ev, battery, bess, charging",Energy,Popper Power GmbH develops advanced energy sto...,2022,...,85741.0,25681.3,53545.9,92300.6,27368.7,57710.0,99178.9,,,106580.3
2,Setter AI,https://www.eu-startups.com/directory/setter-ai/,https://www.eu-startups.com/wp-content/uploads...,https://www.trysetter.com,Wien,Vienna,"ai, ai agents, sales & marketing, ai saas, ai ...",Software & Analytics,Speed matters when you want more sales. That’s...,2024,...,85741.0,25681.3,53545.9,92300.6,27368.7,57710.0,99178.9,,,106580.3
3,SurveySensum,https://www.eu-startups.com/directory/surveyse...,https://www.eu-startups.com/wp-content/uploads...,https://www.surveysensum.com,vienna,Vienna,"customer feedback, customer experience",Software & Analytics,SurveySensum is a leading customer feedback pl...,2018,...,85741.0,25681.3,53545.9,92300.6,27368.7,57710.0,99178.9,,,106580.3
4,Artypa,https://www.eu-startups.com/directory/artypa/,https://www.eu-startups.com/wp-content/uploads...,https://artypa.com,Vienna,Vienna,"ai content, ai productivity, ai generation",Software & Analytics,Traditional AI workflows often involve navigat...,2024,...,85741.0,25681.3,53545.9,92300.6,27368.7,57710.0,99178.9,,,106580.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,Ease-Link,https://www.eu-startups.com/directory/ease-link/,https://www.eu-startups.com/wp-content/uploads...,http://easelink.com/,Graz,Styria,"automotive, industrial automation",Mobility,"Ease-Link GmbH develops MATRIX CHARGING®, the ...",2015,...,43866.4,14076.4,26796.6,46553.2,14962.4,28756.2,51589.0,,,54981.9
514,Healcloud,https://www.eu-startups.com/directory/healcloud/,https://www.eu-startups.com/wp-content/uploads...,http://www.healcloud.com/,Vienna,Vienna,"big data, health care",Software & Analytics,Healcloud is a European health tech enabler to...,2015,...,85741.0,25681.3,53545.9,92300.6,27368.7,57710.0,99178.9,,,106580.3
515,Rendity,https://www.eu-startups.com/directory/rendity/,https://www.eu-startups.com/wp-content/uploads...,https://rendity.com/,Vienna,Vienna,"finance, financial services, real estate",FinTech/InsurTech,Rendity enables real estate investments for ev...,2015,...,85741.0,25681.3,53545.9,92300.6,27368.7,57710.0,99178.9,,,106580.3
516,Ubiq.ai,https://www.eu-startups.com/directory/ubiq-ai/,https://www.eu-startups.com/wp-content/uploads...,https://www.ubiq.ai/,Vienna,Vienna,"autonomous vehicles, car sharing, internet, la...",Mobility,Ubiq is shaping the future of urban mobility b...,2015,...,85741.0,25681.3,53545.9,92300.6,27368.7,57710.0,99178.9,,,106580.3


In [106]:
df_detailed_listings.to_csv("data/eustartup_listings_adjusted_with_economic_vars.csv", index=False)